In [131]:
from scipy.io import loadmat
import sys
import numpy as np
from numpy.fft import *
from scipy.sparse import *

In [169]:
# quickly load all the variables from mat file

thismodule = sys.modules[__name__]
val_dict = loadmat('data/variable_vals_inside_linear_reassignment.mat')

for name, value in val_dict.items():
    if name[:2] == '__': continue
    setattr(thismodule, name, val_dict[name])

In [27]:
# begin reimplementation

In [117]:
N_p = [[x.size]]

In [118]:
assert N_p == N

In [119]:
xf_p = fftn(x)

In [120]:
xf

array([[-0.9788+0.j        ],
       [ 0.4927-0.70927481j],
       [ 0.4927+0.70927481j]])

In [121]:
np.any(xf_p == xf)

True

In [122]:
HT_p = np.ceil(N_p/tdeci)

In [123]:
assert HT_p == HT

In [126]:
noct = noct.astype(np.float32)

In [127]:
HF_p = np.ceil(-noct*np.log2(minf/maxf)+1)

In [129]:
assert HF_p == HF

In [130]:
histo

<1x160 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Column format>

In [165]:
histo_p = csc_matrix((HT[0][0], HF[0][0]))
histc_p = csc_matrix((HT[0][0], HF[0][0]))

In [163]:
histo_p

<1x160 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Column format>

In [160]:
repr(histo)

"<1x160 sparse matrix of type '<class 'numpy.float64'>'\n\twith 0 stored elements in Compressed Sparse Column format>"

In [167]:
assert repr(histo_p) == repr(histo)

In [178]:
f_p = np.arange(0, N) / N

In [180]:
assert np.all(f_p == f)

In [181]:
allt = [];
allf = [];
alle = [];
allc = [];

In [292]:
minf = minf.astype(np.float32)
maxf = maxf.astype(np.float32)
over = over.astype(np.float32)
noct = noct.astype(np.float32)
f = f.astype(np.float32)

In [327]:
# here I manually stepped through the for loop in Matlab and implemented and verified correctness after each step
# all this shows is the implementation is correct over a narrow range of values, but it is a good start

for log2f0 in np.arange(0, HF.astype(np.float32)*over):
    f0 = minf*2**(log2f0/over/noct)
    sigma = f0/(2*np.pi*q)
    gau = np.exp(-(f-f0)**2 / (2*sigma**2))
    gde = -1/sigma**1 * (f-f0) * gau
    
    xi = ifftn(gau.T * xf)
    eta = ifftn(gde.T * xf)
    ener = abs(xi)**2
    
    tins = np.arange(1, N+1).reshape(-1, 1) + np.imag(mp)/(2*np.pi*sigma)
    fins = f0 - np.real(mp)*sigma;
    
    mask = (abs(mp)<lint) & (fins < maxf) & (fins>minf) & (tins>=1) & (tins<N)
    tins = tins[mask]
    fins = fins[mask]
    ener = ener[mask]
    
    itins = np.round(tins/tdeci+0.5)               # <---- I am not sure about this line and the next,
    ifins = np.round(-noct*np.log2(fins/maxf)+1)   # check here first if issues found on broader range of vals
    
    # another problematic point in the implementation follows
    if itins.size != 0: allt.append(itins)
    if ifins.size != 0: allf.append(ifins)
    if ener.size != 0: alle.append(itins)
    if itins.size != 0: allc.append(0*itins+1)
    
    # untested
    if(len(allt)>MAXL):
        histo = hstack(histo, csc_matrix(alle, (allt,allf), (HT,HF)))
        histc = hstack(histc, csc_matrix(allc, (allt,allf), (HT,HF)))
        allt = []
        allf = []
        alle = []
        allc = [] 

In [325]:
hstack()

<function scipy.sparse.construct.hstack(blocks, format=None, dtype=None)>

In [316]:
allt.append(itins)

In [317]:
allt

[array([], shape=(1, 0), dtype=float64)]

In [313]:
ifins

array([], shape=(1, 0), dtype=float64)

In [309]:
itins

array([], shape=(1, 0), dtype=float64)

In [307]:
tins

array([], dtype=float64)

In [300]:
itins

array([[1.],
       [1.],
       [1.]])

array([], dtype=float64)

In [295]:
mask = (abs(mp)<lint) & (fins < maxf) & (fins>minf) & (tins>=1) & (tins<N)

In [296]:
mask

array([[False],
       [False],
       [False]])

In [284]:
fins

array([[3.46944695e-18],
       [3.46944695e-18],
       [3.46944695e-18]])

In [283]:
f0

array([[0.02]])

In [278]:
np.real(mp)*sigma

array([[0.02],
       [0.02],
       [0.02]])

In [274]:
(np.real(mp)*sigma)

dtype('float64')

In [272]:
fins

array([[3.46944695e-18],
       [3.46944695e-18],
       [3.46944695e-18]])

In [270]:
f0

array([[0.02]], dtype=float32)

In [269]:
np.real(mp)*sigma

array([[0.02],
       [0.02],
       [0.02]])

In [267]:
fins

array([[3.46944695e-18],
       [3.46944695e-18],
       [3.46944695e-18]])

In [259]:
(abs(mp)<lint) & (fins < maxf) & (fins>minf) & (tins>=1) & (tins<N)

ValueError: operands could not be broadcast together with shapes (3,1) (0,1) 

In [260]:
mp < lint

array([[False],
       [False],
       [False]])

In [262]:
fins

array([], shape=(0, 1), dtype=uint8)

In [261]:
fins < maxf

array([], shape=(0, 1), dtype=bool)

array([[1.],
       [2.],
       [3.]])

In [251]:
tins

array([], shape=(0, 1), dtype=uint8)

In [250]:
ener

array([[7.61879273e-19],
       [7.61879273e-19],
       [7.61879273e-19]])

In [247]:
mp

array([[6.28318531-0.j],
       [6.28318531-0.j],
       [6.28318531-0.j]])

In [248]:
ene

array([], shape=(0, 1), dtype=uint8)

array([[-5.48432203e-09+0.j],
       [-5.48432203e-09+0.j],
       [-5.48432203e-09+0.j]])

In [242]:
xi

array([[-8.7285696e-10+0.j],
       [-8.7285696e-10+0.j],
       [-8.7285696e-10+0.j]])

In [234]:
gde

array([[ 1.68093237e-08, -0.00000000e+00, -0.00000000e+00]])

In [217]:
sigma

array([[0.0031831]])

NameError: name 'pi' is not defined

array([[2.67528696e-09, 0.00000000e+00, 0.00000000e+00]])

In [221]:
f.dtype

dtype('<f8')

In [215]:
f

array([[0.        , 0.33333333, 0.66666667]])